<a href="https://colab.research.google.com/github/rmanzi13/Entrenador_Personal/blob/main/Entrenamiento_Personal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df_ejercicio = pd.read_csv('rutinas_ejercicio.csv')
df_alimentacion = pd.read_csv('planes_alimentacion.csv')

## Creamos mapeos para indices únicos

In [4]:
### Primero usando df_ejercicio

unique_rou_name = set(df_ejercicio['rou_name'])
rou_name_mapping = {name: i for i, name in enumerate(unique_rou_name)}

unique_rou_type = set(df_ejercicio['rou_type'])
rou_type_mapping = {tipo: i for i, tipo in enumerate(unique_rou_type)}

unique_rou_time = set(df_ejercicio['rou_time'])
rou_time_mapping = {time: i for i, time in enumerate(unique_rou_time)}

unique_rou_intensity = set(df_ejercicio['rou_intensity'])
rou_intensity_mapping = {intensity: i for i, intensity in enumerate(unique_rou_intensity)}

unique_rou_level = set(df_ejercicio['rou_level'])
rou_level_mapping = {level: i for i, level in enumerate(unique_rou_level)}

unique_rou_objectives = set(df_ejercicio['rou_objectives'])
rou_objectives_mapping = {objectives: i for i, objectives in enumerate(unique_rou_objectives)}

unique_rou_descrip = set(df_ejercicio['rou_descrip'])
rou_descrip_mapping = {descr: i for i, descr in enumerate(unique_rou_descrip)}

In [14]:
# Usando df_alimentacion

unique_food_name = set(df_alimentacion['food_name'])
food_name_mapping = {name: i for i, name in enumerate(unique_food_name)}

unique_food_diet = set(df_alimentacion['food_diet'])
food_diet_mapping = {diet: i for i, diet in enumerate(unique_food_diet)}

unique_food_hours = set(df_alimentacion['food_hours'])
food_hours_mapping = {hours: i for i, hours in enumerate(unique_food_hours)}

unique_food_recipes = set(df_alimentacion['food_recipes'])
food_recipes_mapping = {rec: i for i, rec in enumerate(unique_food_recipes)}

unique_food_objectives = set(df_alimentacion['food_objectives'])
food_objectives_mapping = {obj: i for i, obj in enumerate(unique_food_objectives)}

unique_food_description = set(df_alimentacion['food_description'])
food_description_mapping = {descr: i for i, descr in enumerate(unique_food_description)}



## Aplicamos mapeos a DataFrame

In [8]:
## Aplicación a DataFrame df_ejercicio

df_ejercicio['rou_name_num'] = df_ejercicio['rou_name'].map(rou_name_mapping)
df_ejercicio['rou_type_num'] = df_ejercicio['rou_type'].map(rou_type_mapping)
df_ejercicio['rou_time_num'] = df_ejercicio['rou_time'].map(rou_time_mapping)
df_ejercicio['rou_intensity_num'] = df_ejercicio['rou_intensity'].map(rou_intensity_mapping)
df_ejercicio['rou_level_num'] = df_ejercicio['rou_level'].map(rou_level_mapping)
df_ejercicio['rou_objectives_num'] = df_ejercicio['rou_objectives'].map(rou_objectives_mapping)
df_ejercicio['rou_descrip_num'] = df_ejercicio['rou_descrip'].map(rou_descrip_mapping)

In [15]:
## Aplicación a DataFrame df_alimentacion
df_alimentacion['food_name_num'] = df_alimentacion['food_name'].map(food_name_mapping)
df_alimentacion['food_diet_num'] = df_alimentacion['food_diet'].map(food_diet_mapping)
df_alimentacion['food_hours_num'] = df_alimentacion['food_hours'].map(food_hours_mapping)
df_alimentacion['food_recipes_num'] = df_alimentacion['food_recipes'].map(food_recipes_mapping)
df_alimentacion['food_objectives'] = df_alimentacion['food_objectives'].map(food_objectives_mapping)
df_alimentacion['food_description_num'] = df_alimentacion['food_description'].map(food_description_mapping)

In [16]:
# Realizamos el merge
df_combined = pd.merge(df_ejercicio[['rou_descrip_num', 'rou_descrip']], df_alimentacion[['food_name_num', 'food_description']], left_index=True, right_index=True)

In [27]:
df_combined.head(10)

,rou_descrip_num,rou_descrip,food_name_num,food_description
0,8,Rutina enfocada en el desarrollo muscular usan...,5,Plan diseñado para complementar rutinas de ent...
1,9,Ejercicios cardiovasculares para mejorar la ca...,7,Plan nutricional equilibrado para mantener una...
2,4,Ejercicios para mejorar el rango de movimiento...,3,Dieta baja en calorías diseñada para apoyar la...
3,2,Rutina de alta intensidad con ejercicios varia...,6,"Dieta vegana equilibrada, rica en nutrientes y..."
4,0,Sesiones de yoga enfocadas en la flexibilidad ...,8,Dieta cetogénica centrada en la reducción de c...
5,1,Ejercicios centrados en fortalecer los músculo...,6,Comidas a base de plantas para aquellos que si...
6,5,"Ejercicios para mejorar la agilidad, velocidad...",2,Comidas sin gluten para aquellos con restricci...
7,7,Ejercicios de larga duración y baja intensidad...,1,Comidas sin lácteos para aquellos con restricc...
8,6,Ejercicios para mejorar el equilibrio y la est...,4,Comidas altas en fibra para mejorar la digestión.
9,3,Ejercicios de alta intensidad para mejorar la ...,0,Comidas bajas en sodio para aquellos con restr...


### Modelo

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# Leemos las preguntas del formulario desde el archivo de texto
with open("formulario.txt", "r") as file:
    formulario = file.readlines()

In [20]:
# Procesamos texto para descripciones y preguntas
vectorizer = TfidfVectorizer()
descripciones_codificadas = vectorizer.fit_transform(df_combined['rou_descrip'].astype(str) + ' ' + df_combined['food_description'].astype(str))
preguntas_codificadas = vectorizer.transform(formulario)

In [21]:
# Calculamos similitud del coseno
similarity_matrix = cosine_similarity(descripciones_codificadas, preguntas_codificadas)

In [25]:
# Obtener recomendaciones
umbral_similitud = 0.1
for i, pregunta in enumerate(formulario):
    similitudes_pregunta = similarity_matrix[:, i]
    filas_recomendadas = df_combined[similitudes_pregunta > umbral_similitud]
    print(f"\nRecomendaciones para la pregunta {i} ({pregunta}):")
    print(filas_recomendadas[['rou_descrip', 'food_name_num', 'food_description']])


Recomendaciones para la pregunta 0 (﻿1. ¿Tu cuerpo tiende a acumular mucha grasa?
):
                                         rou_descrip  food_name_num  \
2  Ejercicios para mejorar el rango de movimiento...              3   
3  Rutina de alta intensidad con ejercicios varia...              6   

                                    food_description  
2  Dieta baja en calorías diseñada para apoyar la...  
3  Dieta vegana equilibrada, rica en nutrientes y...  

Recomendaciones para la pregunta 1 (2. ¿Cómo describirías la forma de tu cuerpo? ¿Redonda y suave (hombres) o en forma de pera (mujeres)?
):
                                         rou_descrip  food_name_num  \
0  Rutina enfocada en el desarrollo muscular usan...              5   
2  Ejercicios para mejorar el rango de movimiento...              3   
3  Rutina de alta intensidad con ejercicios varia...              6   
4  Sesiones de yoga enfocadas en la flexibilidad ...              8   
5  Ejercicios centrados en fortalecer 